In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
sys.path.append('./waveglow/')

import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils

# load WaveGlow
waveglow_path = '/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/tacotron2/waveglow/waveglow_256channels_universal_v4.pt' # or change to the latest version of the pretrained WaveGlow.
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()
from apex import amp
waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.

/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/glow-tts/.env/lib/python3.8/site-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/glow-tts/.env/lib/python3.8/site-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/glow-tts/.env/lib/python3.8/site-packages/torch/serialization.py:888: SourceChangeWarning: so

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/glow-tts/.env/lib/python3.8/site-packages/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)


In [12]:
# If you are using your own trained model
model_dir = "logs/base_blank_ms_old"
hps = utils.get_hparams_from_dir(model_dir)
checkpoint_path = utils.latest_checkpoint_path(model_dir)

# If you are using a provided pretrained model
# hps = utils.get_hparams_from_file("./configs/any_config_file.json")
# checkpoint_path = "/path/to/pretrained_model"

model = models.FlowGenerator(
    len(symbols) + getattr(hps.data, "add_blank", False),
    out_channels=hps.data.n_mel_channels,
    n_speakers=hps.data.n_speakers,
    **hps.model).to("cuda")

utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

# cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int16")

logs/base_blank_ms_old/G_36.pth


In [24]:
tst_stn = "So maybe I should put my heart out on the table" 

if getattr(hps.data, "add_blank", False):
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'])
    text_norm = commons.intersperse(text_norm, len(symbols))
else: # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
    tst_stn = " " + tst_stn.strip() + " "
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'])
sequence = np.array(text_norm)[None, :]
print("".join([symbols[c] if c < len(symbols) else "<BNK>" for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

<BNK>s<BNK>o<BNK> <BNK>m<BNK>a<BNK>y<BNK>b<BNK>e<BNK> <BNK>i<BNK> <BNK>s<BNK>h<BNK>o<BNK>u<BNK>l<BNK>d<BNK> <BNK>p<BNK>u<BNK>t<BNK> <BNK>m<BNK>y<BNK> <BNK>h<BNK>e<BNK>a<BNK>r<BNK>t<BNK> <BNK>o<BNK>u<BNK>t<BNK> <BNK>o<BNK>n<BNK> <BNK>t<BNK>h<BNK>e<BNK> <BNK>t<BNK>a<BNK>b<BNK>l<BNK>e<BNK>


In [26]:
with torch.no_grad():
  noise_scale = .667
  length_scale = 1.0
  sid = torch.LongTensor([2]).cuda()
  (y_gen_tst, *_), *_, (attn_gen, *_) = model(x_tst, x_tst_lengths, g=sid, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  try:
    audio = waveglow.infer(y_gen_tst.half(), sigma=.666)
  except:
    audio = waveglow.infer(y_gen_tst, sigma=.666)
ipd.Audio(normalize_audio(audio[0].clamp(-1,1).data.cpu().float().numpy()), rate=hps.data.sampling_rate)